<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/Real_state_url_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re,os
import time
from datetime import datetime
import matplotlib.dates as mdates
import seaborn as sns
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
for dirname, _, filenames in os.walk('/content/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
def get_data(url):  
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    real_estate_data = soup.find_all("dl", class_='dl-horizontal-border')
    is_property_found = "Yes"
    data_dict = {}
    data_dict["URL"] = url
    if real_estate_data:
        for d in real_estate_data:
            dt = d.find_all('dt')
            dd = d.find_all('dd')
            
            for i,j in zip(dt, dd):
                i = i.contents[0].strip()
                j = j.contents[0].strip()
                if i == "Unit Number":
                    if j == "-":
                        data_dict["Unit Number"] = np.nan
                    else:
                        data_dict["Unit Number"] = j
                elif i == "Price":
                    data_dict["Price(¥)"] = float(j.replace(",","").replace("¥","").strip())
                elif i == "Building Name":
                    data_dict["Building Name"] = j
                elif i == "Available From":
                    if "Please Inquire" in j:
                        data_dict["Available From"] = np.nan
                    else:
                        data_dict["Available From"] = datetime.strptime(j, '%b %d, %Y')
                elif i == "Type":
                    data_dict["Type"] = j.replace(" ", "")
                elif i == "Size":
                    data_dict["Size(m²)"] = float(j.replace("m²", "").replace(",", "").strip())
                elif i == "Gross Yield":
                    data_dict["Gross Yield(%)"] = float(j.replace("%", "").strip())
                elif i == "Land Rights":
                    data_dict["Land Rights"] = j
                elif i == "Maintenance Fee":
                    data_dict["Maintenance Fee(¥/mnt)"] = float(j.replace("¥", "").replace(" / mth", "").strip().replace(",",""))
                elif i == "Location":
                    data_dict["Location"] = j.replace(",", "")
                elif i == "Occupancy":
                    data_dict["Occupancy"] = j
                elif i == "Floor":
                    data_dict["Floor"] = j.replace(" ", "")
                elif i == "Nearest Station":
                    data_dict["Nearest Station"] = j.split("(")[0].strip()
                    if len(j.split("(")) > 1:
                        if "walk" in j:
                            data_dict["Way to Nearest Station"] = "Walk"
                            data_dict["Distance From Station(min)"] = j.split("(")[1].split("min")[0].strip()
                        elif "bus" in j:
                            data_dict["Way to Nearest Station"] = "Bus"
                            data_dict["Distance From Station(min)"] = j.split("(")[1].split("min")[0].strip()
                elif i == "Layout":
                    data_dict["Layout"] = j
                elif i == "Year Built":
                    data_dict["Year Built"] = j
                elif i == "Direction Facing":
                    data_dict["Direction Facing"] = j.replace(",", "")
                elif i == "Transaction Type":
                    data_dict["Transaction Type"] = j
                elif i == "Balcony Size":
                    data_dict["Balcony Size(m²)"] = float(j.replace("m²", "").replace(",", "").strip())
                elif i == "Building Description":
                    data_dict["Building Description"] = j.replace(",", "")
                elif i == "Other Expenses":
                    j = j.replace(",", "").replace(" ", "").replace("，", "")
                    lst = re.findall(r'\d+', j)
                    if len(lst) > 0:
                        lst = [int(i) for i in lst] 
                        data_dict["Other Expenses"] = sum(lst)
                elif i == "Parking":
                    data_dict["Parking Available"] = j.split()[0].replace(",", "")
                    if len(j.split()) > 1:
                        if j.split()[0].replace(",", "") == "Available":
                            data_dict["Parking Fee(¥/mnt)"] = float(j.split()[1].replace(",", "").replace("¥", "").strip())
                elif i == "Date Updated":
                    if "Please Inquire" in j:
                        data_dict["Date Updated"] = np.nan
                    else:
                        data_dict["Date Updated"] = datetime.strptime(j, '%b %d, %Y')
                elif i == "Next Update Schedule":
                    if "Please Inquire" in j:
                        data_dict["Next Update Schedule"] = np.nan
                    else:
                        data_dict["Next Update Schedule"] = datetime.strptime(j, '%b %d, %Y')
                    
    else:
        is_property_found = "No"
    data_dict["Is_Prop_Avl"] = is_property_found
    return data_dict

## Data Cleaning

In [3]:
import requests
from bs4 import BeautifulSoup
import re

url = "https://www.google.com/search?q=املاک+تهران+اجاره"
r = requests.get(url)
page = r.text 
soup = BeautifulSoup(page, 'lxml') 

i = 0

link_list = []
for tag in soup.find_all('a'):
    i+=1
    href = tag['href']
    if re.search('http',href):
        try:
            link = re.search('https://.+\.com',href).group(0)
            link_list.append(link)
        except:
            pass

link_list = list(set(link_list))

link_list2 = [] 

for link in link_list:
    if not re.search('google.com',link):
        link_list2.append(link)
        
print(link_list2)

import pandas as pd

df = pd.DataFrame(link_list2)
df.to_csv('/content/sample_data/bs4_final.csv', index=False)

['https://shabesh.com', 'https://www.sheypoor.com', 'https://www.zoomila.com', 'https://www.2nabsh.com', 'https://kilid.com']


In [4]:
df = pd.read_csv("/content/sample_data/bs4_final.csv" , delim_whitespace=True)#"/kaggle/input/japanese-property-urls/tokyo_property_urls.csv")
print(df)

                          0
0       https://shabesh.com
1  https://www.sheypoor.com
2   https://www.zoomila.com
3    https://www.2nabsh.com
4         https://kilid.com


In [19]:
urls = df.columns[0:]
len(urls)
print (urls)

Index(['0'], dtype='object')


## Scrape Data

In [9]:
real_estate_df = pd.DataFrame(columns=["URL", "مناسب برای", "شماره واحد", "قیمت", "نام ساختمان", "طبقه", "در دسترس از", "نوع", "اندازه", "بازده ناخالص",
                                      "حقوق زمین", "هزینه نگهداری", "موقعیت مکانی", "شغل", "نزدیکترین ایستگاه", "راه تا نزدیکترین ایستگاه", "فاصله از ایستگاه (دقیقه",
                                      "نما", "سال ساخت", "رو به جهت", "نوع معامله", "اندازه بالکن (متر مربع)", "شرح ساختمان", "سایر هزینه ها",
                                      "پارکینگ موجود", "هزینه پارکینگ", "برنامه به‌روزرسانی بعدی", "برنامه به‌روزرسانی بعدی"])
for url in urls:
    print (url)
    # res = get_data(url)
    # real_estate_df = real_estate_df.append(res, ignore_index=True)

0
